In [1]:
import pandas as pd 
import numpy as np
import csv
import baseline_functions as base

# restore saved variables
%store -r summary_violent6_fl_model

no stored variable summary_violent6_fl_model


In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_violence_six_month'].values
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_violence_six_month'].values

### Model

In [3]:
#### Logistic
c = [1e-5, 1e-4, 1e-3, 1e-2]
logistic_summary = base.Logistic(FL_X, FL_Y, KY_X, KY_Y,C=c,seed=816)

#### Lasso
c = [1e-5, 1e-4, 1e-3, 1e-2]
lasso_summary = base.Lasso(FL_X, FL_Y, KY_X, KY_Y,C=c,seed=816)

#### LinearSVM
c = [1e-5, 1e-4, 1e-3, 1e-2]
svm_summary = base.LinearSVM(FL_X, FL_Y, KY_X, KY_Y,C=c,seed=816)

#### Random Forest
n_estimators =  [60,80,100]
depth = [1,2]
impurity = [0.001, 0.003, 0.005]
rf_summary = base.RF(FL_X, FL_Y, KY_X, KY_Y, 
                     depth=depth, 
                     estimators=n_estimators, 
                     impurity=impurity,
                     seed=816)

#### XGBoost
learning_rate = [0.05]
depth = [1,2]
n_estimators = [40,60,80]
gamma = [8,10,12]
child_weight = [8,10,12]
subsample = [0.3, 0.5]
xgb_summary = base.XGB(FL_X, FL_Y, KY_X, KY_Y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       gamma=gamma, 
                       child_weight=child_weight, 
                       subsample=subsample, 
                       seed=816)

#### save results
summary_violent6_fl_model = {"Logistic": logistic_summary,
                                    "Lasso": lasso_summary,
                                    "LinearSVM": svm_summary,
                                    "RF": rf_summary,
                                    "XGBoost": xgb_summary}
%store summary_violent6_fl_model

UsageError: Unknown variable 'summary_violent6_fl_model'


In [5]:
results = []

for model_name, model_summary in summary_violent6_fl_model.items():
    results.append([model_name, 
                    model_summary['best_auc'], 
                    model_summary['auc_diff'], 
                    model_summary['best_param'], 
                    model_summary['KY_score']])
results

[['Logistic',
  0.6646544991345431,
  0.06636587449661258,
  {'C': 0.01},
  0.7523346176956779],
 ['Lasso',
  0.650172966527686,
  0.02557902128336087,
  {'C': 0.01},
  0.7692634252004606],
 ['LinearSVM',
  0.6188601877566065,
  0.06500540847105163,
  {'C': 0.01},
  0.6161761913545994],
 ['RF',
  0.6783688386884116,
  0.030733341538477288,
  {'max_depth': 1, 'min_impurity_decrease': 0.001, 'n_estimators': 80},
  0.8040848868764867],
 ['XGBoost',
  0.6954382080637694,
  0.016882102916320085,
  {'gamma': 8,
   'learning_rate': 0.05,
   'max_depth': 2,
   'min_child_weight': 8,
   'n_estimators': 60,
   'subsample': 0.3},
  0.8029130185653209]]

In [7]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\Six Month\\"
results = [["Violent",             
            round(logistic_summary['KY_score'],3), 
            round(lasso_summary['KY_score'], 3), 
            round(svm_summary['KY_score'],3), 
            round(rf_summary['KY_score'],3), 
            round(xgb_summary['KY_score'],3)]]
with open(path + 'Six Month FL_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)